In [2]:
import json
import pandas as pd
import re 
import glob 
import copy 
import os 
import nltk
import unicodedata 
from collections import Counter
from nltk.collocations import *

C:\Users\mrpie\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import spacy

In [3]:
# !pip install -U pip setuptools wheel
# !pip install -U spacy
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
#!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()
from bs4 import BeautifulSoup
from html import unescape

In [ ]:
%%time 
### Creating reviews dataframe 
data_file = open("yelp_academic_dataset_review.json",encoding='utf-8')
data = []
for line in data_file:
    data.append(json.loads(line))
reviewsdf = pd.DataFrame(data)
data_file.close()

In [3]:
### Creataing businesses dataframe 
data_file = open("yelp_academic_dataset_business.json",encoding='utf-8')
data = []
for line in data_file:
    data.append(json.loads(line))
businessdf = pd.DataFrame(data)
data_file.close()

### For now, use smaller version of reviewsdf just to make sure everything works

In [6]:
reviewsdf = reviewsdf.head(10000)

### Generate hashtags from reviews df

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [8]:
#Adjust mindf parameter
mindf = int(len(reviewsdf) * .001)
tfidf = TfidfVectorizer(sublinear_tf=True,
               analyzer='word',
#                max_features=2000,
               tokenizer=word_tokenize,
               stop_words=nltk.corpus.stopwords.words("english"),min_df=5)

In [9]:
text = list(reviewsdf['text'])
text = [re.sub(r'[^A-Za-z]+'," ",document) for document in text]


In [10]:
%%time
hashtagdf = pd.DataFrame(tfidf.fit_transform(text).toarray())

C:\Users\mrpie\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


Wall time: 38.3 s


In [11]:
hashtagdf.columns = tfidf.get_feature_names()

C:\Users\mrpie\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
hashtagdfsum = hashtagdf.sum()
tfidf_scores = [(i,hashtagdfsum[i]) for i in hashtagdfsum.index]
tfidf_scores.sort(key=lambda x : x[1],reverse=True)

In [67]:
##REMOVE?
lower_threshold = int(len(tfidf_scores) * .08)
upper_threshold = int(len(tfidf_scores) * .60)
scores = tfidf_scores[lower_threshold:]
tfidf_dict = {item[0] : item[1] for item in scores}

In [68]:
#Maybe add ability to get N most useful tags 
def generate_tags(text):
    tags = set()
    for s in text.split(' '):
        if s in tfidf_dict:
            tags.add(s)
    return list(tags)[:5]

In [69]:
reviewsdf['tags'] = reviewsdf['text'].apply(generate_tags)

In [70]:
reviewsdf.head(5)

,review_id,text,business_id,tags
0,KU_O5udG6zpxOg-VcAEodg,"If you decide to eat here, just be aware it is...",XQfwVwDr-v0ZS3_CbbE5Xw,"[opt, takes, spent, experiences, aware]"
1,BiTunyQ73aT9WBnpR9DZGw,I've taken a lot of spin classes over the year...,7ATYjTIgM3jUlt4UM3IypQ,"[desire, compares, spin, giving, recommendations]"
2,saUsX_uimxRlCVr67Z4Jig,Family diner. Had the buffet. Eclectic assortm...,YjUWPpI6HXG530lwP-fb2A,"[choices, relaxed, casual, rolled, grape]"
3,AqPFMleE6RsU23_auESxiA,"Wow! Yummy, different, delicious. Our favo...",kxX2SOes4o-D3ZQBkiMRfA,"[kinds, curry, deter, changed, lamb]"
4,Sx8TMOWLNuJBWer-0pcmoA,Cute interior and owner (?) gave us tour of up...,e4Vwtrqf-wpJfwesgvdgxQ,"[esp, draft, upcoming, w, limited]"


In [17]:
# trigram_measures = nltk.collocations.TrigramAssocMeasures()


In [18]:
# finder = BigramCollocationFinder.from_words()
# finder1 = finder.apply_freq_filter(1)
# finder.nbest(bigram_measures.pmi, 1500)
# #Maybe add bigrams to each string, so they end upo in freq dict
# # list(bigrams)

### Hashtag generation above needs to be improved, just temporarlily using it to continue with other parts

# Create graph data

1 ) Create  weighted edges between business and associated hashtags (edge weight is number of reviews that contained particular hashtag?)

2) Create weighted edges between hashtags that occur in the same business (or review ?) weight = num co coccurances in reviews?

Drop unecessary columns from reviewsdf 

In [71]:
reviewsdf = reviewsdf[['review_id','text','business_id','tags']]
reviewsdf

,review_id,text,business_id,tags
0,KU_O5udG6zpxOg-VcAEodg,"If you decide to eat here, just be aware it is...",XQfwVwDr-v0ZS3_CbbE5Xw,"[opt, takes, spent, experiences, aware]"
1,BiTunyQ73aT9WBnpR9DZGw,I've taken a lot of spin classes over the year...,7ATYjTIgM3jUlt4UM3IypQ,"[desire, compares, spin, giving, recommendations]"
2,saUsX_uimxRlCVr67Z4Jig,Family diner. Had the buffet. Eclectic assortm...,YjUWPpI6HXG530lwP-fb2A,"[choices, relaxed, casual, rolled, grape]"
3,AqPFMleE6RsU23_auESxiA,"Wow! Yummy, different, delicious. Our favo...",kxX2SOes4o-D3ZQBkiMRfA,"[kinds, curry, deter, changed, lamb]"
4,Sx8TMOWLNuJBWer-0pcmoA,Cute interior and owner (?) gave us tour of up...,e4Vwtrqf-wpJfwesgvdgxQ,"[esp, draft, upcoming, w, limited]"
...,...,...,...,...
9995,ZcBtCA9jGhLfakf1jJ2BAg,Excellent food and service. The place is funct...,1-z7wd860Rii4kbEMCT8DA,"[environment, welcoming, funky, choices]"
9996,UIkEO-10J6Y99IhRqUflvg,Just about to get tucked into a meatloaf that ...,EpREWeEpmR8f1qLHzzF0AA,"[smelling, meatloaf, tucked]"
9997,S-NQM3Axcg8JS3MXHUIvyw,Outstanding customer service! And my car is dr...,dvidzWEPgTQPeBc8CUV2OQ,"[showed, taken, auto, talk, human]"
9998,ME79YrEhm2xe4IQy_0zkGw,I and my husband went here for Dinner one day ...,2XYPFRm7teCUr3eGsB2-qw,"[along, environment, spring, closer, noise]"


In [75]:
###Create graph between business and associated hashtags 
from collections import defaultdict
businessAndTagGraph = defaultdict(lambda: defaultdict(int))
tagsGraph = defaultdict(lambda: defaultdict(int))

In [76]:
%%time
for review in reviewsdf.itertuples():
    businessId = review[3]
    tags = review[4]
    for i in range(len(tags)): 
        #Update Edge weights (number of times hashtag occurs in review)
        currTag = tags[i]
        businessAndTagGraph[businessId][currTag] += 1
        #Create edges and weights for tags in same review 
        for j in range(len(tags)):
            if i != j:
                 tagsGraph[currTag][tags[j]] += 1

Wall time: 164 ms


#### Save graphs as csv before loading into neo4j?

# Load graph data into neo4j

1) Finish cretaing graphs in python - Zafeer

2) Create Graphs in Neo4j - Tony

3) Maybe work a bit on hash tags - Terin 

4) 2 Applications each All

5) Add redis to cache queries All  

6) Create powerpoint presentation  

### Meet @ 5pm Thursday

### Application Ideas

### Tony
1) Find geographically close businesses that share the same/similar hashtags given a business id 

2) Find businesses in a given city that have similar hashtags to a given business.

### Zafeer

3) Find the N highest rated business in a city that contain a specific hashtag

4) Find top 10 hashtags that are associated with a specific business id 

### Terin
5) Given a certain category, what are the top hashtags for that category

6) N most common hashtags in a particular city